In [4]:
import pandas as pd 
import psycopg2
from prophet import Prophet

In [5]:
CONNECTION = "dbname=postgres user=postgres password=horus123 host=localhost port=15432"
SELECT_QUERY = "select time_bucket('5 min', time) as time_5min, last(closing_price, time) as closing_price\
    from ohlcvs where exchange='bittrex' and symbol='BTC-USD' and time < '2020-01-01'\
    group by time_5min order by time_5min asc;"
COLUMN_NAMES = ["ds", "y"]

def psql_to_df(connection_string, query, column_names):
    conn = psycopg2.connect(connection_string)
    with conn:
        cur = conn.cursor()
        cur.execute(query)
        rows = cur.fetchall()
        df = pd.DataFrame(rows, columns=column_names)
        return df

In [6]:
df = psql_to_df(CONNECTION, SELECT_QUERY, COLUMN_NAMES)

In [7]:
df['ds'] = df['ds'].dt.tz_localize(None)

In [8]:
df.head()

,ds,y
0,2019-01-01 00:00:00,3687.901
1,2019-01-01 00:05:00,3683.788
2,2019-01-01 00:10:00,3685.137
3,2019-01-01 00:15:00,3688.504
4,2019-01-01 00:20:00,3685.581


In [9]:
df.shape

(103680, 2)

In [ ]:
model = Prophet(
    daily_seasonality=False, 
    seasonality_mode="multiplicative",
)
metrics = model.fit(df)
future = model.make_future_dataframe(periods=12)
forecast = model.predict(future)
fig1 = model.plot(forecast)

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
